Test case for handling overlap mesages

In [136]:
## function copied from v3.3
import numpy as np

verbose = True


def update_feed_handle_overlap(target_feed, incoming_ids, incoming_shares):
    """
    Update feed with new messages.
    Handle overlapping: if message already exists, reset age to 0. Increment age for the rest of the messages
    New messages are appended to the beginning of the feed, existing messages order is maintained
    - target_feed (tuple of lists): (message_ids, no_shares, ages), represents an agent's news feed
    """

    messages, no_shares, ages = [np.array(i) for i in target_feed]
    # return sorted intersection
    overlap, x_ind, y_ind = np.intersect1d(messages, incoming_ids, return_indices=True)
    # print(new_messages, new_shares, new_ages)
    if verbose:
        print(f"incoming ids : {incoming_ids} --> feed: {messages}")
        print(f"no_shares : {incoming_shares} --> feed: {no_shares}")
        print(f"incoming age : {np.zeros(len(incoming_ids))} --> feed: {ages}")

        print(f"overlap message between {messages} and {incoming_ids} are: {overlap}")
        print(f"sorted messages incoming: {sorted_incoming_messages}")
        print(
            f"no_shares: old: {new_shares[x_ind]}, incoming: {sorted_incoming_shares[y_ind]}"
        )

        print("update no_share and age of overlapping messages.. ")
        print(f"before:  messages: {messages}, shares: {no_shares}, ages: {ages}")

    print("incoming except for overlap: ", incoming_ids[~np.array(y_ind)])
    print(
        "incoming except for overlap (tuple): ", incoming_ids[tuple(~np.array(y_ind))]
    )
    print("x_ind, y_ind: ", x_ind, y_ind)
    # mask_y = np.zeros(len(incoming_ids), bool)
    # mask_y[[y_ind]] = True

    # print("(correct) incoming except for overlap: ", incoming_ids[~mask_y])
    # mask_x = np.zeros(len(messages), bool)
    # mask_x[[x_ind]] = True

    mask_x, mask_y = np.zeros(len(messages), bool), np.zeros(len(incoming_ids), bool)
    mask_x[[x_ind]], mask_y[[y_ind]] = True, True

    print("(correct) incoming except for overlap: ", incoming_ids[~mask_y])

    print(f"mask_x: {mask_x}, mask_y: {mask_y}")
    # update no_shares and age of existing messages from the sorted old feed:
    print(
        "incoming_shares (to add to existing) incoming[mask_y]: ",
        incoming_shares[mask_y],
    )
    print("no_shares (existing) [mask_x]: ", no_shares[mask_x])
    no_shares[mask_x] += incoming_shares[mask_y]

    # add age to all other messages
    ages += np.ones(len(ages), dtype=int)
    # reset age existing messages
    ages[mask_x] = np.zeros(len(y_ind))

    if verbose:
        print(f"after:  messages: {messages}, shares: {no_shares}, ages: {ages}")
    # push new messages into the feed (only the non-overlapping messages)
    no_shares = np.insert(no_shares, 0, incoming_shares[~mask_y])
    messages = np.insert(messages, 0, incoming_ids[~mask_y])

    ages = np.insert(ages, 0, np.zeros(len(incoming_shares[~mask_y])))

    if verbose:
        print(f"updated: messages: {messages}, shares: {no_shares}, ages: {ages}")
    updated_feed = (list(messages), list(no_shares), list(ages))
    return updated_feed


def _handle_oversized_feed(newsfeed, alpha=15):
    """
    Handles oversized newsfeed
    Returns the newsfeed (tuple of lists) where the oldest message is removed
    Input:
        feed (tuple - (list of int, list of int)): (list of mess_ids - list of popularities), represents an agent's news feed
    """
    # Remove messages based on age - oldest first
    messages, shares, ages = newsfeed

    sorted_by_age = sorted(zip(messages, shares, ages), key=lambda x: x[2])
    sorted_messages, sorted_shares, sorted_ages = [list(i) for i in zip(*sorted_by_age)]

    updated_feed = (
        sorted_messages[:alpha],
        sorted_shares[:alpha],
        sorted_ages[:alpha],
    )

    assert len(updated_feed[0]) <= alpha

    return updated_feed

In [5]:
import numpy as np

incoming_ids = np.array([100, 2])
incoming_shares = np.array([10, 4])

### Test case 1 

In [7]:
# incoming messages, if not overlapping are added at the beginning of the list
# if a message overlaps, its popularity is updated (+=new popularity), and age is reset to 0
messages = np.array([3, 1, 2, 4])
no_shares = [1, 1, 3, 1]
ages = [1, 1, 1, 1]

target_feed = (messages, no_shares, ages)
correct_output = ([100, 1, 2, 3, 4], [10, 1, 7, 1, 1], [0, 2, 0, 2, 2])

In [8]:
new_feed = _update_feed_handle_overlap(target_feed, incoming_ids, incoming_shares)

incoming ids : [100   2] --> feed: [3 1 2 4]
no_shares : [10  4] --> feed: [1, 1, 3, 1]
incoming age : [0. 0.] --> feed: [1, 1, 1, 1]
overlap message between [3 1 2 4] and [100   2] are: [2]
sorted messages incoming: [  2 100]
no_shares: old: [7], incoming: [4]
update no_share and age of overlapping messages.. 
before:  messages: [1 2 3 4], shares: [1 7 1 1], ages: [2 0 2 2]
updated: messages: [100   1   2   3   4], shares: [10  1  7  1  1], ages: [0 2 0 2 2]


In [11]:
for idx, alist in enumerate(correct_output):
    print(alist)
    assert new_feed[idx] == alist

[100, 1, 2, 3, 4]
[10, 1, 7, 1, 1]
[0, 2, 0, 2, 2]


In [12]:
clipped = _handle_oversized_feed(new_feed, alpha=3)
clipped

([100, 2, 1], [10, 7, 1], [0, 0, 2])

In [13]:
correct_output_oversize = ([100, 2, 1], [10, 7, 1], [0, 0, 2])
for idx, alist in enumerate(correct_output_oversize):
    print(alist)
    assert clipped[idx] == alist

[100, 2, 1]
[10, 7, 1]
[0, 0, 2]


### Test case 1 

In [137]:
# incoming messages, if not overlapping are added at the beginning of the list
# if a message overlaps, its popularity is updated (+=new popularity), and age is reset to 0
messages = np.array([3, 1, 2, 4])
no_shares = [1, 1, 3, 1]
ages = [1, 1, 1, 1]

target_feed = (messages, no_shares, ages)
incoming_ids = np.array([100, 2])
incoming_shares = np.array([10, 4])

print(f"incoming ids : {incoming_ids} --> feed: {messages}")
print(f"no_shares : {incoming_shares} --> feed: {no_shares}")
print(f"incoming age : {np.zeros(len(incoming_ids))} --> feed: {ages}")

incoming ids : [100   2] --> feed: [3 1 2 4]
no_shares : [10  4] --> feed: [1, 1, 3, 1]
incoming age : [0. 0.] --> feed: [1, 1, 1, 1]


In [138]:
correct_output = ([100, 3, 1, 2, 4], [10, 1, 1, 7, 1], [0, 2, 2, 0, 2])

In [139]:
new_feed = update_feed_handle_overlap(target_feed, incoming_ids, incoming_shares)
for idx, alist in enumerate(correct_output):
    print(alist)
    assert new_feed[idx] == alist

incoming ids : [100   2] --> feed: [3 1 2 4]
no_shares : [10  4] --> feed: [1 1 3 1]
incoming age : [0. 0.] --> feed: [1 1 1 1]
overlap message between [3 1 2 4] and [100   2] are: [2]
update no_share and age of overlapping messages.. 
before:  messages: [3 1 2 4], shares: [1 1 3 1], ages: [1 1 1 1]
incoming except for overlap:  [100]
incoming except for overlap (tuple):  100
x_ind, y_ind:  [2] [1]
(correct) incoming except for overlap:  [100]
mask_x: [False False  True False], mask_y: [False  True]
incoming_shares (to add to existing) incoming[mask_y]:  [4]
no_shares (existing) [mask_x]:  [3]
after:  messages: [3 1 2 4], shares: [1 1 7 1], ages: [2 2 0 2]
updated: messages: [100   3   1   2   4], shares: [10  1  1  7  1], ages: [0 2 2 0 2]
[100, 3, 1, 2, 4]
[10, 1, 1, 7, 1]
[0, 2, 2, 0, 2]


/var/folders/gj/zfw5qy_n09d78gdzsfht4c7m0000gn/T/ipykernel_91285/3125993726.py:39: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  mask_x[[x_ind]], mask_y[[y_ind]] = True, True


### Test case 2

In [97]:
# incoming messages, if not overlapping are added at the beginning of the list
# if a message overlaps, its popularity is updated (+=new popularity), and age is reset to 0
messages = np.array([1053])
no_shares = [1]
ages = [1]

target_feed = (messages, no_shares, ages)

incoming_ids = np.array([2878, 2108, 1053, 2629])
incoming_shares = np.array([1, 1, 1, 1])

print(f"incoming ids : {incoming_ids} --> feed: {messages}")
print(f"no_shares : {incoming_shares} --> feed: {no_shares}")
print(f"incoming age : {np.zeros(len(incoming_ids))} --> feed: {ages}")

incoming ids : [2878 2108 1053 2629] --> feed: [1053]
no_shares : [1 1 1 1] --> feed: [1]
incoming age : [0. 0. 0. 0.] --> feed: [1]


In [98]:
correct_output = ([2878, 2108, 2629, 1053], [1, 1, 1, 2], [0, 0, 0, 0])

In [99]:
new_feed = update_feed_handle_overlap(target_feed, incoming_ids, incoming_shares)
for idx, alist in enumerate(correct_output):
    print(alist)
    assert new_feed[idx] == alist

incoming ids : [2878 2108 1053 2629] --> feed: [1053]
no_shares : [1 1 1 1] --> feed: [1]
incoming age : [0. 0. 0. 0.] --> feed: [1]
overlap message between [1053] and [2878 2108 1053 2629] are: [1053]
update no_share and age of overlapping messages.. 
before:  messages: [1053], shares: [1], ages: [1]
incoming except for overlap:  [2108]
incoming except for overlap (tuple):  2108
(correct) incoming except for overlap:  [2878 2108 2629]
mask_x: [ True], mask_y: [False False  True False]
incoming_shares (to add to existing) incoming[mask_y]:  [1]
no_shares (existing) [mask_x]:  [1]
after:  messages: [1053], shares: [2], ages: [0]
updated: messages: [2878 2108 2629 1053], shares: [1 1 1 2], ages: [0 0 0 0]
[2878, 2108, 2629, 1053]
[1, 1, 1, 2]
[0, 0, 0, 0]


/var/folders/gj/zfw5qy_n09d78gdzsfht4c7m0000gn/T/ipykernel_91285/2184657156.py:38: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  mask_x[[x_ind]], mask_y[[y_ind]] = True, True


### Try to fix Warning but it doesn't work

In [ ]:
# overlap, x_ind, y_ind = np.intersect1d(messages, incoming_ids, return_indices=True)
# mask_x, mask_y = np.zeros(len(messages), bool), np.zeros(len(incoming_ids), bool)
mask_x, mask_y = np.zeros(4, bool), np.zeros(7, bool)
x_ind, y_ind = np.array([2, 3]), np.array([0, 1])
mask_x[tuple(x_ind)], mask_y[tuple(y_ind)] = True, True
# mask_x = tuple(mask_x)

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [ ]:
np.zeros(4, bool)

array([False, False, False, False])

In [ ]:
messages[~mask_x]

array([3, 1])

In [ ]:
mask_y = np.zeros(len(incoming_ids), bool)
mask_y[[y_ind]] = True